In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt.pyll.base import scope


import mag
from mag.experiment import Experiment
from mag import summarize


from speech2phone.preprocessing import get_TIMIT, get_phones
from speech2phone.preprocessing.filters import mel

np.set_printoptions(precision=3)

# Random Forest with `mag`

In [3]:
X_toy, y_toy = get_TIMIT(dataset='toy', preprocessor=mel, TIMIT_root='../TIMIT/TIMIT')

Loading toy/mel set from cache... done.


In [4]:
def random_forest_acc(n_estimators, max_depth):
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_toy, y_toy)
    acc = rf.score(X_toy, y_toy)
    return acc



experiments_dir = '../experiments/results/random_forest_v1'
config = {
    'n_estimators': 11,
    'max_depth': 10
}
with Experiment(config=config, experiments_dir=experiments_dir) as experiment:
    config = experiment.config
    acc = random_forest_acc(config.n_estimators, config.max_depth)
    
    experiment.register_result('accuracy', acc)

ValueError: Experiment with identifier 10|11 already exists. Set `resume_from` to the corresponding identifier (directory name) 10|11 or delete it manually and then rerun the code.

# Random Forest with `hyperopt`

In [5]:
def acc_wrapper(args):
    acc = random_forest_acc(args['n_estimators'], args['max_depth'])
    print("{}: {:.3f}".format(args, acc))
    return -acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=50)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))

{'max_depth': 20, 'n_estimators': 4}: 0.903
{'max_depth': 1, 'n_estimators': 99}: 0.137
{'max_depth': 13, 'n_estimators': 61}: 0.982
{'max_depth': 5, 'n_estimators': 98}: 0.321
{'max_depth': 15, 'n_estimators': 56}: 0.998
{'max_depth': 6, 'n_estimators': 10}: 0.374
{'max_depth': 14, 'n_estimators': 41}: 0.992
{'max_depth': 9, 'n_estimators': 34}: 0.672
{'max_depth': 5, 'n_estimators': 16}: 0.312
{'max_depth': 7, 'n_estimators': 77}: 0.465
{'max_depth': 25, 'n_estimators': 68}: 1.000
{'max_depth': 25, 'n_estimators': 39}: 1.000
{'max_depth': 2, 'n_estimators': 74}: 0.145
{'max_depth': 6, 'n_estimators': 6}: 0.350
{'max_depth': 13, 'n_estimators': 87}: 0.985
{'max_depth': 29, 'n_estimators': 35}: 1.000
{'max_depth': 24, 'n_estimators': 82}: 1.000
{'max_depth': 21, 'n_estimators': 49}: 1.000
{'max_depth': 29, 'n_estimators': 40}: 1.000
{'max_depth': 19, 'n_estimators': 21}: 1.000
{'max_depth': 19, 'n_estimators': 22}: 1.000
{'max_depth': 29, 'n_estimators': 64}: 1.000
{'max_depth': 25, 'n

# Random Forest with `mag` and `hyperopt`

In [6]:
def acc_wrapper(args):
    experiments_dir = '../experiments/results/random_forest_v2'
    with Experiment(config=args, experiments_dir=experiments_dir) as experiment:
        config = experiment.config
        acc = random_forest_acc(config.n_estimators, config.max_depth)
        experiment.register_result('accuracy', acc)
    return -acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=5)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))


Best Raw: {'max_depth': 22.532586122234616, 'n_estimators': 41.60007385667237}
Best Readable: {'max_depth': 22, 'n_estimators': 41}
